In [76]:
%cd D:/archive/food-101

D:\archive\food-101


In [77]:
import pandas as pd
train_df = pd.read_csv('./meta/train.txt', header = None, names=['path'])
test_df = pd.read_csv('./meta/test.txt', header = None, names=['path'])

In [78]:
def spliter(data, class_or_id='id'):
    if class_or_id.upper() == 'CLASS':
        output = data.split('/')[0]
        
    else:
        output = data.split('/')[-1]
    return output

In [79]:
train_df['label'] = train_df['path'].map(lambda x: spliter(data = x, class_or_id = 'Class')) 
train_df.head()

,path,label
0,apple_pie/1005649,apple_pie
1,apple_pie/1014775,apple_pie
2,apple_pie/1026328,apple_pie
3,apple_pie/1028787,apple_pie
4,apple_pie/1043283,apple_pie


In [80]:
train_df['idx'] = train_df['path'].map(lambda x: spliter(x)) 
train_df.head()

,path,label,idx
0,apple_pie/1005649,apple_pie,1005649
1,apple_pie/1014775,apple_pie,1014775
2,apple_pie/1026328,apple_pie,1026328
3,apple_pie/1028787,apple_pie,1028787
4,apple_pie/1043283,apple_pie,1043283


In [81]:
test_df['label'] = test_df['path'].map(lambda x: spliter(x, 'class')) 
test_df['idx'] = test_df['path'].map(lambda x: spliter(x)) 
test_df.head()

,path,label,idx
0,apple_pie/1011328,apple_pie,1011328
1,apple_pie/101251,apple_pie,101251
2,apple_pie/1034399,apple_pie,1034399
3,apple_pie/103801,apple_pie,103801
4,apple_pie/1038694,apple_pie,1038694


In [82]:
#20개만 갖고 할게요
mini_20 = train_df['label'].unique()[:20]
mini_20 = [f.upper() for f in mini_20]
mini_20

['APPLE_PIE',
 'BABY_BACK_RIBS',
 'BAKLAVA',
 'BEEF_CARPACCIO',
 'BEEF_TARTARE',
 'BEET_SALAD',
 'BEIGNETS',
 'BIBIMBAP',
 'BREAD_PUDDING',
 'BREAKFAST_BURRITO',
 'BRUSCHETTA',
 'CAESAR_SALAD',
 'CANNOLI',
 'CAPRESE_SALAD',
 'CARROT_CAKE',
 'CEVICHE',
 'CHEESECAKE',
 'CHEESE_PLATE',
 'CHICKEN_CURRY',
 'CHICKEN_QUESADILLA']

In [83]:
def prepare_data(label):
    if label.upper() in mini_20:
        return label
    else:
        return 'Others'

In [84]:
train_df['label'] = train_df['label'].map(lambda x: prepare_data(x))
test_df['label'] = test_df['label'].map(lambda x: prepare_data(x))

In [85]:
train_df['label'].unique()

array(['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio',
       'beef_tartare', 'beet_salad', 'beignets', 'bibimbap',
       'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad',
       'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake',
       'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'Others'],
      dtype=object)

In [86]:
def add_exact_path(row):
    relative_path = row['path']
    if 'food-101/images' not in relative_path:
        return os.path.join('food-101/images', relative_path)
    else:
        return relative_path


In [87]:
train_df['path'] = train_df[['path']].apply(add_exact_path, axis=1)
test_df['path'] = test_df[['path']].apply(add_exact_path, axis=1)
train_df.head()

,path,label,idx
0,food-101/images\apple_pie/1005649,apple_pie,1005649
1,food-101/images\apple_pie/1014775,apple_pie,1014775
2,food-101/images\apple_pie/1026328,apple_pie,1026328
3,food-101/images\apple_pie/1028787,apple_pie,1028787
4,food-101/images\apple_pie/1043283,apple_pie,1043283


## Convert Tensorflow to Pytorch

In [88]:
import torch
from torch.optim import lr_scheduler  # Add this line
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image

In [89]:
class FoodDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join('./food-101/images', self.df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [90]:
# Create test dataset and dataloader
test_dataset = FoodDataset(df=test_df, transform=data_transforms['val'])
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=4)

In [91]:
from torch.utils.data import DataLoader
from torchvision import transforms

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create datasets
image_datasets = {x: FoodDataset(df=train_df if x=='train' else valid_df if x=='val' else test_df, transform=data_transforms[x])
                for x in ['train', 'val']}

# Create dataloaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
                for x in ['train', 'val']}


In [92]:
print(train_df.head())
print(valid_df.head())
print(test_df.head())

                                path      label      idx
0  food-101/images\apple_pie/1005649  apple_pie  1005649
1  food-101/images\apple_pie/1014775  apple_pie  1014775
2  food-101/images\apple_pie/1026328  apple_pie  1026328
3  food-101/images\apple_pie/1028787  apple_pie  1028787
4  food-101/images\apple_pie/1043283  apple_pie  1043283
                                              path   label      idx
18388         food-101/images\clam_chowder/2871728  Others  2871728
47035  food-101/images\macaroni_and_cheese/3545493  Others  3545493
72532                food-101/images\tacos/3633647  Others  3633647
47016  food-101/images\macaroni_and_cheese/3453546  Others  3453546
46354  food-101/images\lobster_roll_sandwich/69108  Others    69108
                                path      label      idx
0  food-101/images\apple_pie/1011328  apple_pie  1011328
1   food-101/images\apple_pie/101251  apple_pie   101251
2  food-101/images\apple_pie/1034399  apple_pie  1034399
3   food-101/images\ap

In [93]:
# Data augmentation and normalization for training
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [94]:
# Assuming dataframe for the dataset
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)
test_df = test_df

image_datasets = {
    'train': CustomDataset(train_df, transform=data_transforms['train']),
    'val': CustomDataset(valid_df, transform=data_transforms['val']),
    'test': CustomDataset(test_df, transform=data_transforms['test'])
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=0)
    for x in ['train', 'val', 'test']
}

In [95]:
# Define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a new network
model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features

c:\Users\user\miniconda3\envs\nic\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\nic\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [96]:
# Here the size of each output sample is set to 21.
model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 21),
    nn.LogSoftmax(dim=1)
)

In [97]:
# Move the model to the device
model_ft = model_ft.to(device)

In [98]:
# Define the loss function
criterion = nn.NLLLoss()

In [99]:
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)

In [100]:
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [101]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        print()

    return model

In [102]:
def evaluate(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [104]:
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, num_epochs=30)

Epoch 0/29
----------


FileNotFoundError: [Errno 2] No such file or directory: 'food-101/images\\panna_cotta/2680719'

In [ ]:
# Evaluate the model
result = evaluate(model_ft, dataloaders['test'])
print(result)

NameError: name 'evaluate' is not defined